# Load and downsample

In [35]:
import rasterio
import numpy as np
from rasterio.enums import Resampling
from rasterio.transform import Affine, rowcol

# Paths
tif_path = '/Applications/Emmanuel Zheng/habsim/data/gmted_tif/mn30_grd.tif'
npy_path = '/Applications/Emmanuel Zheng/habsim/data/worldelev.npy'

# Open TIFF and downsample by factor of 2 using bilinear interpolation
with rasterio.open(tif_path) as src:
    data = src.read(
        1,  # first band
        out_shape=(src.height // 2, src.width // 2),
        resampling=Resampling.bilinear
    ).astype(np.float32)
    
    # Update transform to match downsampled array
    transform = src.transform * Affine.scale(src.width / data.shape[1], src.height / data.shape[0])

# Save downsampled array
np.save(npy_path, data, allow_pickle=False)
print(f"Saved downsampled array with shape {data.shape} to {npy_path}")

Saved downsampled array with shape (10440, 21600) to /Applications/Emmanuel Zheng/habsim/data/worldelev.npy


# Get bounds for .npy reading

In [42]:
import rasterio

path = '/Applications/Emmanuel Zheng/habsim/data/gmted_tif/mn30_grd.tif'

with rasterio.open(path) as src:
    print("Bounds:", src.bounds)
    print("Width (cols):", src.width)
    print("Height (rows):", src.height)
    print("Resolution (lon_deg, lat_deg):", src.res)
    print("CRS:", src.crs)


Bounds: BoundingBox(left=-180.00013888888893, bottom=-90.0001388888889, right=179.99985967111152, top=83.99986041511133)
Width (cols): 43200
Height (rows): 20880
Resolution (lon_deg, lat_deg): (0.008333333300000011, 0.008333333300000011)
CRS: EPSG:4326


# Test

In [43]:
import numpy as np

# Preload once (memory-mapped for speed)
data = np.load('/Applications/Emmanuel Zheng/habsim/data/worldelev.npy', mmap_mode='r')
rows, cols = data.shape

# Use bounds from raster metadata
MIN_LON = -180.00013888888893
MAX_LON = 179.99985967111152
MAX_LAT = 83.99986041511133
MIN_LAT = -90.0001388888889

def getElevation(lat, lon):
    """Return bilinearly interpolated elevation for (lat, lon)."""
    # Clamp input to data bounds
    lat = np.clip(lat, MIN_LAT, MAX_LAT)
    lon = ((lon + 180) % 360) - 180  # normalize

    # Compute fractional column/row
    col_f = (lon - MIN_LON) / (MAX_LON - MIN_LON) * (cols - 1)
    row_f = (MAX_LAT - lat) / (MAX_LAT - MIN_LAT) * (rows - 1)

    # Integer indices and fractions
    x0 = int(np.floor(col_f))
    y0 = int(np.floor(row_f))
    x1 = min(x0 + 1, cols - 1)
    y1 = min(y0 + 1, rows - 1)
    fx = col_f - x0
    fy = row_f - y0

    # Bilinear interpolation
    v00 = data[y0, x0]
    v10 = data[y0, x1]
    v01 = data[y1, x0]
    v11 = data[y1, x1]
    v_top = v00 * (1 - fx) + v10 * fx
    v_bottom = v01 * (1 - fx) + v11 * fx
    elev = v_top * (1 - fy) + v_bottom * fy

    return float(max(0, elev))

lat, lon = 37.4393, -121.5769  # near Fremont, CA
print(getElevation(lat, lon))


684.1148914641363
